In [1]:
import requests
import json    # 파이썬 기본 모듈
import urllib  # 파이썬 기본 모듈 (URLEncoding 기능 수행)
import os  
import datetime as dt

In [2]:
# OpenAPI 인증키 정의

api_key = 'f4a8121ce977a43eee8fc5de78c18f5f'

In [3]:
# Kakao API에게 전달할 요청 파라미터 정의

q = "파이썬"    # 검색어
page = 1       # 접근할 페이지 번호(1~50)
size = 80      # 가져올 데이터 수 (1~80)

In [4]:
# 이미지가 저장될 폴더 이름 문자열 만들기

datetime = dt.datetime.now().strftime("%y%m%d_%H%M%S")
dirname = "%s_%s" % (q, datetime)
dirname

'파이썬_210728_121519'

In [5]:
# 이미지가 저장될 폴더 생성하기
if not os.path.exists(dirname):
    os.mkdir(dirname)

In [6]:
# 접속에 필요한 헤더정보 구성하기

user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"
header_info = {'User-agent': user_agent, 'referer': None, 'Authorization': 'KakaoAK ' + api_key}
header_info

{'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36',
 'referer': None,
 'Authorization': 'KakaoAK f4a8121ce977a43eee8fc5de78c18f5f'}

#### API를 활용하여 데이터 수집하기

In [7]:
# API 에 전달할 파라미터 인코딩

params = {"page": page, "size": size, "query": q}
query = urllib.parse.urlencode(params)
query

'page=1&size=80&query=%ED%8C%8C%EC%9D%B4%EC%8D%AC'

In [8]:
# 최종 접속 주소 구성

url_tpl = "https://dapi.kakao.com/v2/search/image"
api_url = url_tpl + "?" + query
api_url

'https://dapi.kakao.com/v2/search/image?page=1&size=80&query=%ED%8C%8C%EC%9D%B4%EC%8D%AC'

In [9]:
# API에 접근해서 데이터 수집하기

r = requests.get(api_url, headers=header_info)

# 결과 검사
if r.status_code != 200:
    # 에러코드와 에러메시지를 문자열로 구성
    err_msg = "%d %s 에러가 발생했습니다." % (r.status_code, r.reason)
    # 강제로 에러를 발생시킨다.
    raise Exception(err_msg)
    
r.encoding = "utf-8"
image_dict = json.loads(r.text)
#image_dict

In [10]:
# API결과에서 이미지 URL을 추출한 후 해당 이미지 다운로드 받기

# API가 응답한 내용중에서 검색결과에 해당하는 부분만 추출
image_list = image_dict['documents']

# 이미지 주소에 대해서만 반복
for i, v in enumerate(image_list):
    #print(v)
    #print(v['image_url'])
    
    # 파일이 저장될 경로 생성
    path = "%s/%s_%04d.jpg" % (dirname, q, i)
    print( "[%s] >> %s" % (path, v['image_url']) )
    
    # 원본 이미지를 다운받기 위해서는 doc_url값을 이미지가 포함된 웹 페이지 주소로 지정해야 한다.
    header_info['referer'] = v['doc_url']
        
    # 이미지 주소를 다운로드를 위해 stream 모드로 가져온다.
    r = requests.get(v['image_url'], headers=header_info, stream=True)
    
    # 에러 발생시 저장이 불가능하므로 건너뛰고 반복의 조건식으로 이동
    if r.status_code != 200:
        # 에러코드와 에러메시지를 문자열로 구성
        err_msg = "%d %s 에러가 발생했습니다." % (r.status_code, r.reason)
        print(err_msg)
        continue

    # 예외처리 구문 적용
    try:
        # 추출한 데이터를 저장
        # -> 'w': 텍스트 쓰기 모드, 'wb': 바이너리(이진값) 쓰기 모드
        with open(path, 'wb') as f:
            f.write(r.raw.read())
            print("----------> 저장성공")
    except Exception as e:
        # 이미지 다운로드 실패시 에러가 발생함을 알림
        print( "[Error] `%s`의 저장에 실패했습니다." % path )
        # 개발자가 원인을 파악할 수 있도록 상세 에러 원인 출력
        print(e)
        # 다음 이미지를 시도하기 위해 반복의 조건식으로 이동함
        continue

[파이썬_210728_121519/파이썬_0000.jpg] >> https://k.kakaocdn.net/dn/VN3IA/btqDUE5QTAI/C3c4Iflai0bzJKfpdEBmv1/img.png
----------> 저장성공
[파이썬_210728_121519/파이썬_0001.jpg] >> http://postfiles14.naver.net/MjAxOTA0MDhfMjg0/MDAxNTU0NzE4NzYwNTk5.2_fzuAecwRb2Gp4xgJz3OAqm6e3lTrq5U-otIsvWIbog.MbaXdnlekuvWbO0gLwpmsFaaM3KtXlkB6xIqltaaYKwg.JPEG.ehdalsqkr1/%ED%8C%8C%EC%9D%B4%EC%8D%AC1.jpg?type=w580
----------> 저장성공
[파이썬_210728_121519/파이썬_0002.jpg] >> http://postfiles10.naver.net/MjAxOTA4MDVfMTcx/MDAxNTY0OTk1MzA1NjU2._Kyl0SRi1tdzpyvH70J3VJeOfaMrZASzSGSbJzNiRrkg.BQdPooPRCel96SPktXCHZmSRzU7aUKZ9tcrb6goiu7Qg.JPEG.speak1w111z/%ED%8C%8C%EC%9D%B4%EC%8D%AC%EA%B8%B0%EC%B4%88_%EC%BB%A4%EB%A6%AC%ED%81%98%EB%9F%BC.jpg?type=w580
----------> 저장성공
[파이썬_210728_121519/파이썬_0003.jpg] >> http://postfiles12.naver.net/MjAxOTA4MDVfMTgg/MDAxNTY0OTk1MzA1Njc0.pnqQXrJbeyNWkGn6k8ZXesYWq6TjfPSTMmFF6pAWRkUg.E4OE0EM4irKtPkCa3qMF39UbUiXI3R4EgIMVoyWaJCIg.JPEG.speak1w111z/%ED%8C%8C%EC%9D%B4%EC%8D%AC_%EC%9B%B9%EC%84%9C%EB%B2%84%EA%B5%AC%EC%B

### 반복문을 수행하면서 이미지 수집하기

#### OpenAPI를 통한 전체 검색 결과 수집하기

In [11]:
# 모든 데이터 수집하기

# 추가적인 페이지가 존재하는지 확인하기 위한 변수
is_end = False

# 카카오에 요청할 페이지 번호
page = 1

# 수집결과가 누적될 빈 리스트
image_list = []

# 다음 페이지가 존재하는 동안 반복함
while not is_end:
    # 카카오에 전달할 파라미터 인코딩
    # -> page값이 반복 1회마다 1씩 증가한다.
    params = {"page": page, "size": size, "query": q}
    query = urllib.parse.urlencode(params)

    # 카카오 API 접속 주소 구성
    url_tpl = "https://dapi.kakao.com/v2/search/image"
    api_url = url_tpl + "?" + query

    # 카카오 API 데이터 가져오기
    r = requests.get(api_url, headers=header_info)

    # 연동에 실패한 경우
    if r.status_code != 200:
        # 에러코드와 에러메시지를 문자열로 구성
        err_msg = "%d %s 에러가 발생했습니다." % (r.status_code, r.reason)
        print(err_msg)
        # API 연동 중단
        break

    # 연동 결과를 딕셔너리로 변환
    r.encoding = "utf-8"
    image_dict = json.loads(r.text)
    
    # 카카오 측에서 알려준 이후 페이지 존재 여부 확인
    is_end = image_dict['meta']['is_end']
    
    # 미리 준비한 빈 리스트에 검색 결과만 누적
    image_list += image_dict['documents']
    
    print("%dpage로부터 %d건 수집함 >>> 누적 데이터 수: %d" % (page, len(image_dict['documents']), len(image_list)))
    
    # 다음 반복을 위해 페이지 번호 1 증가
    page += 1
    
    # 카카오 API 스팩상 최대 50페이지 까지만 요청 가능하므로 50 이상이면 반복 중단을 위해 변수값 변경
    if page >= 50:
        is_end = True
    
#print(image_list)

1page로부터 80건 수집함 >>> 누적 데이터 수: 80
2page로부터 80건 수집함 >>> 누적 데이터 수: 160
3page로부터 80건 수집함 >>> 누적 데이터 수: 240
4page로부터 80건 수집함 >>> 누적 데이터 수: 320
5page로부터 80건 수집함 >>> 누적 데이터 수: 400
6page로부터 80건 수집함 >>> 누적 데이터 수: 480
7page로부터 80건 수집함 >>> 누적 데이터 수: 560
8page로부터 80건 수집함 >>> 누적 데이터 수: 640
9page로부터 80건 수집함 >>> 누적 데이터 수: 720
10page로부터 80건 수집함 >>> 누적 데이터 수: 800
11page로부터 80건 수집함 >>> 누적 데이터 수: 880
12page로부터 80건 수집함 >>> 누적 데이터 수: 960
13page로부터 80건 수집함 >>> 누적 데이터 수: 1040
14page로부터 80건 수집함 >>> 누적 데이터 수: 1120
15page로부터 80건 수집함 >>> 누적 데이터 수: 1200
16page로부터 80건 수집함 >>> 누적 데이터 수: 1280
17page로부터 80건 수집함 >>> 누적 데이터 수: 1360
18page로부터 80건 수집함 >>> 누적 데이터 수: 1440
19page로부터 80건 수집함 >>> 누적 데이터 수: 1520
20page로부터 80건 수집함 >>> 누적 데이터 수: 1600
21page로부터 80건 수집함 >>> 누적 데이터 수: 1680
22page로부터 80건 수집함 >>> 누적 데이터 수: 1760
23page로부터 80건 수집함 >>> 누적 데이터 수: 1840
24page로부터 80건 수집함 >>> 누적 데이터 수: 1920
25page로부터 80건 수집함 >>> 누적 데이터 수: 2000
26page로부터 22건 수집함 >>> 누적 데이터 수: 2022


#### 수집된 데이터 셋을 활용하여 이미지 다운로드 받기

In [14]:
# 이미지 다운로드 받기

# 이미지 주소에 대해서만 반복
for i, v in enumerate(image_list):

    # 파일이 저장될 경로 생성
    path = "%s/%s_%04d.jpg" % (dirname, q, i)
    print( "[%s] >> %s" % (path, v['image_url']) )
    
    # 원본 이미지를 다운받기 위해서는 doc_url값을 이미지가 포함된 웹 페이지 주소로 지정해야 한다.
    header_info['referer'] = v['doc_url']
        
    # 이미지 주소를 다운로드를 위해 stream 모드로 가져온다.
    r = requests.get(v['image_url'], headers=header_info, stream=True)
    
    # 에러 발생시 저장이 불가능하므로 건너뛰고 반복의 조건식으로 이동
    if r.status_code != 200:
        # 에러코드와 에러메시지를 문자열로 구성
        err_msg = "%d %s 에러가 발생했습니다." % (r.status_code, r.reason)
        print(err_msg)
        continue
        
    # 예외처리 구문 적용
    try:
        # 추출한 데이터를 저장
        with open(path, 'wb') as f:
            f.write(r.raw.read())
            print("--------> 저장성공")
    except:
        print("~~~~~~~~> 저장실패")
        continue

[파이썬_210728_121519/파이썬_0000.jpg] >> https://k.kakaocdn.net/dn/VN3IA/btqDUE5QTAI/C3c4Iflai0bzJKfpdEBmv1/img.png
--------> 저장성공
[파이썬_210728_121519/파이썬_0001.jpg] >> http://postfiles14.naver.net/MjAxOTA0MDhfMjg0/MDAxNTU0NzE4NzYwNTk5.2_fzuAecwRb2Gp4xgJz3OAqm6e3lTrq5U-otIsvWIbog.MbaXdnlekuvWbO0gLwpmsFaaM3KtXlkB6xIqltaaYKwg.JPEG.ehdalsqkr1/%ED%8C%8C%EC%9D%B4%EC%8D%AC1.jpg?type=w580
--------> 저장성공
[파이썬_210728_121519/파이썬_0002.jpg] >> http://postfiles10.naver.net/MjAxOTA4MDVfMTcx/MDAxNTY0OTk1MzA1NjU2._Kyl0SRi1tdzpyvH70J3VJeOfaMrZASzSGSbJzNiRrkg.BQdPooPRCel96SPktXCHZmSRzU7aUKZ9tcrb6goiu7Qg.JPEG.speak1w111z/%ED%8C%8C%EC%9D%B4%EC%8D%AC%EA%B8%B0%EC%B4%88_%EC%BB%A4%EB%A6%AC%ED%81%98%EB%9F%BC.jpg?type=w580
--------> 저장성공
[파이썬_210728_121519/파이썬_0003.jpg] >> http://postfiles12.naver.net/MjAxOTA4MDVfMTgg/MDAxNTY0OTk1MzA1Njc0.pnqQXrJbeyNWkGn6k8ZXesYWq6TjfPSTMmFF6pAWRkUg.E4OE0EM4irKtPkCa3qMF39UbUiXI3R4EgIMVoyWaJCIg.JPEG.speak1w111z/%ED%8C%8C%EC%9D%B4%EC%8D%AC_%EC%9B%B9%EC%84%9C%EB%B2%84%EA%B5%AC%EC%B6%95.j

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))